In [1]:
import numpy as np
import pandas as pd
import requests
import matplotlib as plt
from bs4 import BeautifulSoup
%matplotlib inline

In [2]:
from bs4 import BeautifulSoup
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(url,"lxml")

In [3]:
df = pd.read_html(url)[0]
df=df[df.Borough != 'Not assigned']
df.rename(index=str, columns={"Neighbourhood": "Neighborhood"}, inplace=True)
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
df = df.groupby(['Postcode','Borough'],as_index=False).agg(lambda x:','.join(x))
df

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [4]:
geoinfo=pd.read_csv('https://cocl.us/Geospatial_data')
df['Latitude'] = df.Postcode.map(geoinfo.set_index('Postal Code')['Latitude'])
df['Longitude'] = df.Postcode.map(geoinfo.set_index('Postal Code')['Longitude'])

In [5]:
from sklearn.preprocessing import StandardScaler
loc = df.values[:,3:5]
loc = np.nan_to_num(loc)
Norloc = StandardScaler().fit_transform(loc)

/Users/Stanley/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/Stanley/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
map_toronto = folium.Map(location=[43.651070,-79.347015], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [7]:
print(df.shape)

(103, 5)
